In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, median_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from scipy.stats import *
import h3
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
rides = pd.read_csv("../big_query_24_14.csv")
rides.rename(columns={'id': 'ride_id'}, inplace=True)
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city
count,2.426000e+04,2.426000e+04,24260.000000,24260.000000,24260.000000,24260.000000,24260.000000,24260.000000,2.426000e+04,24260.0
mean,2.010495e+06,2.376807e+07,35.693980,51.401119,35.694668,51.397562,1003.226051,1236.469992,4.956898e+09,1.0
std,1.440427e+06,1.800928e+07,0.096191,0.143753,0.095899,0.152071,725.298062,1034.221594,3.405802e+05,0.0
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.636840,50.016300,24.000000,120.000000,4.956287e+09,1.0
25%,5.573940e+05,5.043194e+06,35.663319,51.331473,35.661003,51.330028,464.000000,504.000000,4.956594e+09,1.0
50%,1.992264e+06,2.400925e+07,35.723982,51.404408,35.723633,51.401817,799.000000,905.000000,4.956982e+09,1.0
75%,3.323714e+06,3.831664e+07,35.755246,51.465694,35.754699,51.470322,1347.000000,1650.000000,4.957167e+09,1.0
max,4.346548e+06,5.750996e+07,35.839420,52.794230,36.259760,52.792877,7794.000000,9088.000000,4.957626e+09,1.0


In [3]:
columns = ['ride_id','driver_id','a_t_a_result.arrival_a_t_a','a_t_a_result.boarding_a_t_a','a_t_a_result.ride_a_t_a','a_t_a_result.arrival_probe_result.probe.point.lat','a_t_a_result.arrival_probe_result.probe.point.lon','a_t_a_result.arrival_probe_result.probe.timestamp','a_t_a_result.arrival_probe_result.confidence','a_t_a_result.arrival_probe_result.h3_index', 'a_t_a_result.arrival_probe_result.k_ring_level','a_t_a_result.boarding_probe_result.probe.point.lat','a_t_a_result.boarding_probe_result.probe.point.lon','a_t_a_result.boarding_probe_result.probe.timestamp','a_t_a_result.boarding_probe_result.confidence','a_t_a_result.boarding_probe_result.h3_index','a_t_a_result.boarding_probe_result.k_ring_level','a_t_a_result.final_destination_probe_result.probe.point.lat','a_t_a_result.final_destination_probe_result.probe.point.lon','a_t_a_result.final_destination_probe_result.probe.timestamp','a_t_a_result.final_destination_probe_result.confidence','a_t_a_result.final_destination_probe_result.h3_index','a_t_a_result.final_destination_probe_result.k_ring_level','a_t_a_result.destination_probe_result.probe.point.lat','a_t_a_result.destination_probe_result.probe.point.lon','a_t_a_result.destination_probe_result.probe.timestamp','a_t_a_result.destination_probe_result.confidence','a_t_a_result.destination_probe_result.h3_index','a_t_a_result.destination_probe_result.k_ring_level','a_t_a_result.extra_destination_probe_result.probe.point.lat','a_t_a_result.extra_destination_probe_result.probe.point.lon','a_t_a_result.extra_destination_probe_result.probe.timestamp','a_t_a_result.extra_destination_probe_result.confidence','a_t_a_result.extra_destination_probe_result.h3_index','a_t_a_result.extra_destination_probe_result.k_ring_level','pickup_a_d_d_result.distance','pickup_a_d_d_result.confidence','pickup_a_d_d_result.route_ratio','pickup_a_d_d_result.g_p_s_ratio','ride_a_d_d_result.distance','ride_a_d_d_result.confidence','ride_a_d_d_result.route_ratio','ride_a_d_d_result.g_p_s_ratio','total_a_d_d_confidence','in_ride_allotment','e_d_d','clickhouse_time','hash']
khatkesh = pd.read_csv("../khatkesh_14_1.csv", names=columns)
khatkesh.describe()

,ride_id,driver_id,a_t_a_result.arrival_a_t_a,a_t_a_result.boarding_a_t_a,a_t_a_result.ride_a_t_a,a_t_a_result.arrival_probe_result.probe.point.lat,a_t_a_result.arrival_probe_result.probe.point.lon,a_t_a_result.arrival_probe_result.probe.timestamp,a_t_a_result.arrival_probe_result.confidence,a_t_a_result.arrival_probe_result.k_ring_level,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,5.028590e+05,5.028590e+05,502859.000000,5.028590e+05,5.028590e+05,502859.000000,502859.000000,5.028590e+05,502859.000000,502859.000000,...,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,502859.000000,5.028590e+05
mean,4.956607e+09,2.379701e+06,186.070441,2.013937e+16,7.155686e+07,34.603114,51.913875,2.457812e+15,0.734051,1.158275,...,0.000326,0.015392,13.564964,0.857177,0.000125,0.707576,0.867706,0.046938,8.377787,9.240882e+18
std,4.392595e+05,1.344870e+06,146.035298,6.091805e+17,2.135430e+09,2.544448,3.538870,2.129146e+17,0.341657,1.439362,...,0.015817,0.017221,17.250296,0.189048,0.005345,0.344261,0.178898,0.332769,12.547627,5.326126e+18
min,4.873587e+09,1.000000e+00,0.000000,0.000000e+00,1.000000e+00,25.277332,44.829102,1.662865e+09,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.062941e+13
25%,4.956373e+09,1.199125e+06,91.000000,8.000000e+00,4.020000e+02,32.695423,50.836760,1.663160e+09,0.591716,0.000000,...,0.000000,0.005714,4.603000,0.750032,0.000000,0.482860,0.792329,0.000000,2.784000,4.635700e+18
50%,4.956612e+09,2.438832e+06,170.000000,3.600000e+01,6.840000e+02,35.707092,51.409603,1.663161e+09,0.769231,1.000000,...,0.000000,0.011905,8.623000,0.957293,0.000000,0.884615,0.957387,0.000000,5.180000,9.246887e+18
75%,4.956863e+09,3.616538e+06,258.000000,1.240000e+02,1.172000e+03,36.264671,52.524117,1.663163e+09,1.000000,2.000000,...,0.000000,0.020833,16.327728,1.000000,0.000000,0.975610,1.000000,0.000000,9.833000,1.385216e+19
max,4.957263e+09,4.346386e+06,7954.000000,1.844674e+19,6.379876e+10,39.661167,61.168961,1.844674e+19,1.000000,4.000000,...,1.000000,1.000000,1784.723999,1.000000,1.000000,1.000000,1.000000,34.022999,1741.516968,1.844670e+19


In [4]:
rides_khatkesh = pd.merge(rides, khatkesh, on="ride_id")
rides_khatkesh.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,1.506300e+04,1.506300e+04,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,1.506300e+04,15063.0,...,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,15063.000000,1.506300e+04
mean,1.981823e+06,2.391234e+07,35.693123,51.402332,35.694000,51.399399,867.554870,1088.631083,4.956710e+09,1.0,...,0.000208,0.014827,13.534466,0.876051,0.000115,0.738491,0.883905,0.015693,8.121885,9.220426e+18
std,1.436073e+06,1.795970e+07,0.097267,0.146990,0.097062,0.152614,633.272365,951.193802,2.752267e+05,0.0,...,0.011839,0.015742,14.004605,0.174419,0.002896,0.321243,0.164876,0.146355,8.625966,5.339763e+18
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.641853,50.535954,24.000000,120.000000,4.956287e+09,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.232000,3.680090e+14
25%,5.346035e+05,5.211465e+06,35.662340,51.330618,35.660725,51.330080,410.000000,436.000000,4.956454e+09,1.0,...,0.000000,0.005319,4.448000,0.813760,0.000000,0.582005,0.835089,0.000000,2.646000,4.576592e+18
50%,1.940900e+06,2.427381e+07,35.724340,51.404480,35.724358,51.402800,689.000000,775.000000,4.956690e+09,1.0,...,0.000000,0.011236,8.968000,0.965429,0.000000,0.900990,0.963664,0.000000,5.365000,9.171676e+18
75%,3.289352e+06,3.849748e+07,35.754384,51.468884,35.754307,51.472001,1138.000000,1392.000000,4.956964e+09,1.0,...,0.000000,0.020000,17.646500,1.000000,0.000000,0.974576,1.000000,0.000000,10.529500,1.387260e+19
max,4.346206e+06,5.750970e+07,35.832066,52.781403,36.048410,52.792877,5231.000000,6709.000000,4.957246e+09,1.0,...,1.000000,0.500000,159.207718,1.000000,0.204274,1.000000,1.000000,4.430000,116.138000,1.844592e+19


In [5]:
rides = rides_khatkesh[(rides_khatkesh['eta'] > 180) &
                       (rides_khatkesh['eta'] < 10800) &
                       (rides_khatkesh['a_t_a_result.ride_a_t_a'] > 180) &
                       (rides_khatkesh['a_t_a_result.ride_a_t_a'] < 10800)]
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,1.431400e+04,1.431400e+04,14314.000000,14314.000000,14314.000000,14314.000000,14314.000000,14314.000000,1.431400e+04,14314.0,...,14314.000000,14314.000000,14314.000000,14314.000000,14314.000000,14314.000000,14314.000000,14314.000000,14314.000000,1.431400e+04
mean,1.980299e+06,2.372658e+07,35.694601,51.400947,35.695543,51.397840,902.663337,1132.744516,4.956706e+09,1.0,...,0.000074,0.013369,14.138394,0.878035,0.000087,0.740149,0.886102,0.015762,8.475427,9.206828e+18
std,1.437208e+06,1.795372e+07,0.095782,0.142208,0.095551,0.148314,628.103121,952.000888,2.738054e+05,0.0,...,0.002724,0.010659,14.102775,0.168137,0.001919,0.319382,0.158786,0.147553,8.689244,5.339666e+18
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.641853,50.535954,181.000000,120.000000,4.956287e+09,1.0,...,0.000000,0.000000,0.194000,0.015032,0.000000,0.000000,0.062460,0.000000,0.598000,3.680090e+14
25%,5.314055e+05,5.056045e+06,35.664489,51.331163,35.663728,51.330590,444.000000,476.000000,4.956451e+09,1.0,...,0.000000,0.005348,4.956000,0.817145,0.000000,0.589102,0.836708,0.000000,2.954000,4.553286e+18
50%,1.935360e+06,2.401134e+07,35.724758,51.404336,35.724820,51.402598,719.000000,810.500000,4.956685e+09,1.0,...,0.000000,0.010870,9.593000,0.963780,0.000000,0.901639,0.963172,0.000000,5.729500,9.150920e+18
75%,3.291454e+06,3.825565e+07,35.754653,51.466925,35.754650,51.470549,1170.000000,1443.000000,4.956957e+09,1.0,...,0.000000,0.018868,18.395999,1.000000,0.000000,0.973684,1.000000,0.000000,10.868250,1.386922e+19
max,4.346206e+06,5.750927e+07,35.828438,52.773940,36.048410,52.792877,5231.000000,6709.000000,4.957224e+09,1.0,...,0.156830,0.052632,159.207718,1.000000,0.075890,1.000000,1.000000,4.430000,116.138000,1.844592e+19


In [6]:
rides = rides[rides['ride_a_d_d_result.confidence'] > 0.6]
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,1.243500e+04,1.243500e+04,12435.000000,12435.000000,12435.000000,12435.000000,12435.000000,12435.000000,1.243500e+04,12435.0,...,12435.000000,12435.000000,12435.000000,12435.000000,12435.000000,12435.000000,12435.000000,12435.000000,12435.000000,1.243500e+04
mean,1.984667e+06,2.368107e+07,35.694305,51.400390,35.695317,51.397322,896.198633,1122.913470,4.956707e+09,1.0,...,0.000079,0.014224,14.802611,0.932249,0.000045,0.827725,0.936971,0.017408,8.390454,9.226360e+18
std,1.440237e+06,1.796537e+07,0.095661,0.143475,0.095143,0.149095,622.199853,942.565978,2.737140e+05,0.0,...,0.002849,0.010544,14.496755,0.099332,0.001173,0.221447,0.089059,0.155708,8.574203,5.348107e+18
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.641853,50.535954,181.000000,120.000000,4.956287e+09,1.0,...,0.000000,0.000000,0.194000,0.600579,0.000000,0.016854,0.540521,0.000000,0.598000,3.680090e+14
25%,5.351490e+05,5.018804e+06,35.664346,51.330320,35.663137,51.329190,443.500000,475.000000,4.956454e+09,1.0,...,0.000000,0.006211,5.310000,0.907555,0.000000,0.766667,0.906503,0.000000,2.951500,4.552304e+18
50%,1.938937e+06,2.397690e+07,35.724570,51.403110,35.724545,51.401720,714.000000,804.000000,4.956685e+09,1.0,...,0.000000,0.011628,10.170000,0.978110,0.000000,0.924528,0.976122,0.000000,5.682000,9.185595e+18
75%,3.300775e+06,3.826664e+07,35.754391,51.466616,35.754288,51.469811,1158.000000,1425.000000,4.956959e+09,1.0,...,0.000000,0.020000,19.242001,1.000000,0.000000,0.979167,1.000000,0.000000,10.783000,1.389922e+19
max,4.346206e+06,5.750927e+07,35.828438,52.773940,36.048410,52.792877,5231.000000,6596.000000,4.957224e+09,1.0,...,0.156830,0.052632,159.207718,1.000000,0.067175,1.000000,1.000000,4.430000,116.138000,1.844592e+19


In [7]:
rides = rides[['eta', 'ata', 'a_t_a_result.ride_a_t_a', 'source_lat', 'source_lng', 'destination_lat', 'destination_lng', 'e_d_d']]

In [8]:
rides['source_h3_4'] = rides.apply(lambda row: h3.geo_to_h3(row.source_lat, row.source_lng, 4), axis=1)
rides['dest_h3_4'] = rides.apply(lambda row: h3.geo_to_h3(row.destination_lat, row.destination_lng, 4), axis=1)

In [10]:
eta_edd_h3 = rides[["eta", "e_d_d", "source_h3_4", "dest_h3_4"]]
ata = rides["a_t_a_result.ride_a_t_a"]

In [11]:
eta_edd_h3_train, eta_edd_h3_test, ata_train, ata_test = train_test_split(eta_edd_h3, ata, test_size=0.05, random_state=73)
print(len(eta_edd_h3_train))
print(len(eta_edd_h3_test))

11813
622


In [12]:
dummy_train_rides = pd.get_dummies(eta_edd_h3_train, prefix={'source_h3_4':'source_h3_4',
                                                       'dest_h3_4':'dest_h3_4'},
                          drop_first=True)
dummy_train_rides.describe()

,eta,e_d_d,source_h3_4_842cd59ffffffff,source_h3_4_842cd5bffffffff,source_h3_4_842cf31ffffffff,source_h3_4_842cf33ffffffff,source_h3_4_842cf35ffffffff,source_h3_4_842cf37ffffffff,source_h3_4_842cf3dffffffff,dest_h3_4_842cd59ffffffff,dest_h3_4_842cd5bffffffff,dest_h3_4_842cf07ffffffff,dest_h3_4_842cf27ffffffff,dest_h3_4_842cf31ffffffff,dest_h3_4_842cf33ffffffff,dest_h3_4_842cf35ffffffff,dest_h3_4_842cf37ffffffff,dest_h3_4_842cf39ffffffff,dest_h3_4_842cf3dffffffff
count,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000
mean,896.309490,8.395520,0.003894,0.000423,0.843139,0.021163,0.054008,0.024972,0.051299,0.003555,0.000423,0.000254,0.000085,0.836367,0.021248,0.050876,0.024719,0.002032,0.059680
std,622.467211,8.574888,0.062283,0.020570,0.363685,0.143934,0.226044,0.156048,0.220617,0.059524,0.020570,0.015935,0.009201,0.369958,0.144215,0.219754,0.155273,0.045030,0.236903
min,181.000000,0.689000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,444.000000,2.956000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,714.000000,5.683000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1156.000000,10.777000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5231.000000,116.138000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
geo_noise = np.count_nonzero(dummy_train_rides, axis=0) < 10

cols = dummy_train_rides.columns
for col, noise in zip(cols, geo_noise):
    if noise:
        # DOESN'T REINDEX
        eta_edd_h3_train = eta_edd_h3_train[dummy_train_rides[col] == 0]
        ata_train = ata_train[dummy_train_rides[col] == 0]

eta_edd_h3_train.describe()

/var/folders/yq/707ysz_j3jlggmtk5clx13rm0000gn/T/ipykernel_46750/592031391.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  eta_edd_h3_train = eta_edd_h3_train[dummy_train_rides[col] == 0]


,eta,e_d_d
count,11804.000000,11804.000000
mean,895.475008,8.373800
std,619.786057,8.472199
min,181.000000,0.689000
25%,444.000000,2.954750
50%,714.000000,5.683000
75%,1156.000000,10.772000
max,5231.000000,116.138000


In [14]:
print(np.count_nonzero(dummy_train_rides, axis=0))
print(dummy_train_rides.columns)

[11813 11813    46     5  9960   250   638   295   606    42     5     3
     1  9880   251   601   292    24   705]
Index(['eta', 'e_d_d', 'source_h3_4_842cd59ffffffff',
       'source_h3_4_842cd5bffffffff', 'source_h3_4_842cf31ffffffff',
       'source_h3_4_842cf33ffffffff', 'source_h3_4_842cf35ffffffff',
       'source_h3_4_842cf37ffffffff', 'source_h3_4_842cf3dffffffff',
       'dest_h3_4_842cd59ffffffff', 'dest_h3_4_842cd5bffffffff',
       'dest_h3_4_842cf07ffffffff', 'dest_h3_4_842cf27ffffffff',
       'dest_h3_4_842cf31ffffffff', 'dest_h3_4_842cf33ffffffff',
       'dest_h3_4_842cf35ffffffff', 'dest_h3_4_842cf37ffffffff',
       'dest_h3_4_842cf39ffffffff', 'dest_h3_4_842cf3dffffffff'],
      dtype='object')


In [15]:
ohe = ColumnTransformer(
    [('OHE', OneHotEncoder(handle_unknown='ignore', sparse=False),['source_h3_4', 'dest_h3_4'])],
    remainder = 'passthrough'
    ).fit(eta_edd_h3_train)

In [16]:
eta_edd_h3_train_transformed = ohe.transform(eta_edd_h3_train)

In [17]:
eta_edd_h3_test_transformed = ohe.transform(eta_edd_h3_test)

In [18]:
reg = LinearRegression().fit(eta_edd_h3_train_transformed, ata_train)

In [19]:
print(reg.coef_)
print(reg.intercept_)

[ -25.75887343  224.94473332  -19.01559462   89.69035555 -118.84195984
  -79.42441842  -71.59424257  117.64382932 -155.64796364   39.22240546
 -111.21523618   62.08345633  121.00354353  -72.53872651   -0.55130831
    1.53506348  -12.22265237]
-161.57924243144953


In [20]:
ata_train_pred = reg.predict(eta_edd_h3_train_transformed)
ata_test_pred = reg.predict(eta_edd_h3_test_transformed)

In [21]:
eta_edd_h3_train_transformed

array([[0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.79100000e+03, 1.66520004e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.63700000e+03, 1.57670002e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.76800000e+03, 2.09650002e+01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 7.09000000e+02, 5.75500011e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.09800000e+03, 5.74200010e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 4.83000000e+02, 3.08599997e+00]])

In [22]:
print("ETA train MAE", mean_absolute_error(ata_train, eta_edd_h3_train_transformed[:, 15]))
print("prediction train MAE", mean_absolute_error(ata_train, ata_train_pred))

print("ETA test MAE ", mean_absolute_error(ata_test, eta_edd_h3_test_transformed[:, 15]))
print("prediction test MAE", mean_absolute_error(ata_test, ata_test_pred))


print("ETA train MSE", mean_squared_error(ata_train, eta_edd_h3_train_transformed[:, 15]))
print("prediction train MSE", mean_squared_error(ata_train, ata_train_pred))

print("ETA test MSE ", mean_squared_error(ata_test, eta_edd_h3_test_transformed[:, 15]))
print("prediction test MSE", mean_squared_error(ata_test, ata_test_pred))


print("ETA train explained_variance_score", explained_variance_score(ata_train, eta_edd_h3_train_transformed[:, 15]))
print("prediction train explained_variance_score", explained_variance_score(ata_train, ata_train_pred))

print("ETA test explained_variance_score ", explained_variance_score(ata_test, eta_edd_h3_test_transformed[:, 15]))
print("prediction test explained_variance_score", explained_variance_score(ata_test, ata_test_pred))


print("ETA train median_absolute_error", median_absolute_error(ata_train, eta_edd_h3_train_transformed[:, 15]))
print("prediction train median_absolute_error", median_absolute_error(ata_train, ata_train_pred))

print("ETA test median_absolute_error ", median_absolute_error(ata_test, eta_edd_h3_test_transformed[:, 15]))
print("prediction test median_absolute_error", median_absolute_error(ata_test, ata_test_pred))


print("ETA train r2_score", r2_score(ata_train, eta_edd_h3_train_transformed[:, 15]))
print("prediction train r2_score", r2_score(ata_train, ata_train_pred))

print("ETA test r2_score ", r2_score(ata_test, eta_edd_h3_test_transformed[:, 15]))
print("prediction test r2_score", r2_score(ata_test, ata_test_pred))


print("ETA train MAPE", mean_absolute_percentage_error(ata_train, eta_edd_h3_train_transformed[:, 15]))
print("prediction train MAPE", mean_absolute_percentage_error(ata_train, ata_train_pred))

print("ETA test MAPE ", mean_absolute_percentage_error(ata_test, eta_edd_h3_test_transformed[:, 15]))
print("prediction test MAPE", mean_absolute_percentage_error(ata_test, ata_test_pred))

ETA train MAE 278.5157573703829
prediction train MAE 234.0541220760237
ETA test MAE  284.29421221864953
prediction test MAE 234.43059718522406
ETA train MSE 246574.0464249407
prediction train MSE 136276.85342773492
ETA test MSE  252857.3199356913
prediction test MSE 129113.47477425056
ETA train explained_variance_score 0.7758751853508523
prediction train explained_variance_score 0.8438333836678524
ETA test explained_variance_score  0.7764103810242154
prediction test explained_variance_score 0.8549435481180574
ETA train median_absolute_error 122.5
prediction train median_absolute_error 147.08303153043994
ETA test median_absolute_error  124.0
prediction test median_absolute_error 150.57483889916506
ETA train r2_score 0.7174381889736823
prediction train r2_score 0.8438333836678525
ETA test r2_score  0.7158508788156106
prediction test r2_score 0.8549083712526228
ETA train MAPE 0.21066943334957403
prediction train MAPE 0.2367096356321409
ETA test MAPE  0.21635844837406298
prediction test MA

In [40]:
rides_p4 = rides[rides['eta'] > 1800]
rides_p4

,eta,ata,a_t_a_result.ride_a_t_a,source_lat,source_lng,destination_lat,destination_lng,e_d_d,source_h3_4,dest_h3_4
6,2085,1874,1879,35.780780,51.329643,35.723328,51.423737,17.827000,842cf31ffffffff,842cf31ffffffff
10,1846,2102,2500,35.713715,51.315964,35.805560,51.411820,20.403000,842cf31ffffffff,842cf31ffffffff
15,2913,4069,4036,35.564354,51.237858,35.669804,51.036410,37.376999,842cf35ffffffff,842cf3dffffffff
30,2584,2611,2572,35.475560,51.087612,35.649170,51.344048,47.419998,842cf3dffffffff,842cf31ffffffff
37,2003,3737,3752,35.706070,51.338432,35.716774,51.455433,15.448000,842cf31ffffffff,842cf31ffffffff
...,...,...,...,...,...,...,...,...,...,...
15019,2117,3445,3559,35.783707,51.374203,35.672913,51.265650,26.542000,842cf31ffffffff,842cf31ffffffff
15025,2134,4072,4337,35.777378,51.447437,35.732320,51.333805,17.971001,842cf31ffffffff,842cf31ffffffff
15032,1891,3041,3008,35.710052,51.389904,35.675682,51.483303,12.630000,842cf31ffffffff,842cf31ffffffff
15039,2694,1947,1969,35.704956,51.229440,35.818474,50.953747,35.577999,842cf31ffffffff,842cf3dffffffff


In [41]:
p4_eta_edd_h3 = rides_p4[["eta", "e_d_d", "source_h3_4", "dest_h3_4"]]
p4_ata = rides_p4["a_t_a_result.ride_a_t_a"]

In [42]:
p4_eta_edd_h3_train, p4_eta_edd_h3_test, p4_ata_train, p4_ata_test = train_test_split(p4_eta_edd_h3, p4_ata, test_size=0.05, random_state=73)
print(len(p4_eta_edd_h3_train))
print(len(p4_eta_edd_h3_test))

1090
58


In [44]:
p4_dummy_train_rides = pd.get_dummies(p4_eta_edd_h3_train, prefix={'source_h3_4': 'source_h3_4',
                                                             'dest_h3_4': 'dest_h3_4'},
                                   drop_first=True)
p4_dummy_train_rides.describe()

,eta,e_d_d,source_h3_4_842cf31ffffffff,source_h3_4_842cf33ffffffff,source_h3_4_842cf35ffffffff,source_h3_4_842cf37ffffffff,source_h3_4_842cf3dffffffff,dest_h3_4_842cf27ffffffff,dest_h3_4_842cf31ffffffff,dest_h3_4_842cf33ffffffff,dest_h3_4_842cf35ffffffff,dest_h3_4_842cf37ffffffff,dest_h3_4_842cf39ffffffff,dest_h3_4_842cf3dffffffff
count,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000,1090.000000
mean,2357.476147,27.349061,0.826606,0.017431,0.066972,0.022018,0.065138,0.000917,0.762385,0.015596,0.040367,0.014679,0.023853,0.139450
std,525.858334,13.373414,0.378762,0.130932,0.250089,0.146810,0.246882,0.030289,0.425817,0.123964,0.196909,0.120319,0.152662,0.346574
min,1801.000000,6.048000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1981.000000,17.807750,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2203.500000,23.676000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2568.750000,34.504000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5231.000000,116.138000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [45]:
p4_geo_noise = np.count_nonzero(p4_dummy_train_rides, axis=0) < 10

p4_cols = p4_dummy_train_rides.columns
for col, noise in zip(p4_cols, p4_geo_noise):
    if noise:
        # DOESN'T REINDEX
        p4_eta_edd_h3_train = p4_eta_edd_h3_train[p4_dummy_train_rides[col] == 0]
        p4_ata_train = p4_ata_train[p4_dummy_train_rides[col] == 0]

p4_eta_edd_h3_train.describe()

,eta,e_d_d
count,1089.000000,1089.000000
mean,2355.837466,27.272992
std,523.308234,13.141520
min,1801.000000,6.048000
25%,1981.000000,17.805000
50%,2203.000000,23.663000
75%,2568.000000,34.471001
max,5231.000000,116.138000


In [46]:
print(np.count_nonzero(p4_dummy_train_rides, axis=0))
print(p4_dummy_train_rides.columns)

[1090 1090  901   19   73   24   71    1  831   17   44   16   26  152]
Index(['eta', 'e_d_d', 'source_h3_4_842cf31ffffffff',
       'source_h3_4_842cf33ffffffff', 'source_h3_4_842cf35ffffffff',
       'source_h3_4_842cf37ffffffff', 'source_h3_4_842cf3dffffffff',
       'dest_h3_4_842cf27ffffffff', 'dest_h3_4_842cf31ffffffff',
       'dest_h3_4_842cf33ffffffff', 'dest_h3_4_842cf35ffffffff',
       'dest_h3_4_842cf37ffffffff', 'dest_h3_4_842cf39ffffffff',
       'dest_h3_4_842cf3dffffffff'],
      dtype='object')


In [47]:
p4_ohe = ColumnTransformer(
    [('OHE', OneHotEncoder(handle_unknown='ignore', sparse=False), ['source_h3_4', 'dest_h3_4'])],
    remainder='passthrough'
).fit(p4_eta_edd_h3_train)

In [48]:
p4_eta_edd_h3_train_transformed = p4_ohe.transform(p4_eta_edd_h3_train)
p4_eta_edd_h3_test_transformed = p4_ohe.transform(p4_eta_edd_h3_test)

In [49]:
p4_reg = LinearRegression().fit(p4_eta_edd_h3_train_transformed, p4_ata_train)
print(p4_reg.coef_)
print(p4_reg.intercept_)

[ 633.47733001  237.17766398  -38.66121995 -312.17971403 -218.23673935
 -301.57732067  397.2119181   288.28817461 -308.11913622  -60.03807356
 -117.14415868 -118.23176186  -81.96696239    1.44165079   -4.67415288]
-415.63403749689314


In [50]:
p4_ata_train_pred = p4_reg.predict(p4_eta_edd_h3_train_transformed)
p4_ata_test_pred = p4_reg.predict(p4_eta_edd_h3_test_transformed)

In [51]:
p4_eta_edd_h3_train_transformed

array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.49900000e+03, 4.10480003e+01],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.23700000e+03, 4.52099991e+01],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 2.06700000e+03, 3.12380009e+01],
       ...,
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.39600000e+03, 5.25239983e+01],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.37800000e+03, 2.30039997e+01],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.43200000e+03, 2.25939999e+01]])

In [52]:
print("ETA train MAE", mean_absolute_error(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train MAE", mean_absolute_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MAE ", mean_absolute_error(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test MAE", mean_absolute_error(p4_ata_test, p4_ata_test_pred))

print("ETA train MSE", mean_squared_error(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train MSE", mean_squared_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MSE ", mean_squared_error(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test MSE", mean_squared_error(p4_ata_test, p4_ata_test_pred))

print("ETA train explained_variance_score", explained_variance_score(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train explained_variance_score", explained_variance_score(p4_ata_train, p4_ata_train_pred))

print("ETA test explained_variance_score ", explained_variance_score(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test explained_variance_score", explained_variance_score(p4_ata_test, p4_ata_test_pred))

print("ETA train median_absolute_error", median_absolute_error(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train median_absolute_error", median_absolute_error(p4_ata_train, p4_ata_train_pred))

print("ETA test median_absolute_error ", median_absolute_error(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test median_absolute_error", median_absolute_error(p4_ata_test, p4_ata_test_pred))

print("ETA train r2_score", r2_score(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train r2_score", r2_score(p4_ata_train, p4_ata_train_pred))

print("ETA test r2_score ", r2_score(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test r2_score", r2_score(p4_ata_test, p4_ata_test_pred))

print("ETA train MAPE", mean_absolute_percentage_error(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train MAPE", mean_absolute_percentage_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MAPE ", mean_absolute_percentage_error(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test MAPE", mean_absolute_percentage_error(p4_ata_test, p4_ata_test_pred))

ETA train MAE 893.9357208448117
prediction train MAE 531.4817678260782
ETA test MAE  921.3965517241379
prediction test MAE 686.751834832589
ETA train MSE 1276766.2203856749
prediction train MSE 481447.75653419277
ETA test MSE  1433092.4655172413
prediction test MSE 804401.4516166177
ETA train explained_variance_score 0.38833909844007064
prediction train explained_variance_score 0.4738247845946859
ETA test explained_variance_score  0.21839702919065196
prediction test explained_variance_score 0.26677186431703703
ETA train median_absolute_error 730.0
prediction train median_absolute_error 424.77749194156377
ETA test median_absolute_error  694.0
prediction test median_absolute_error 585.620734480683
ETA train r2_score -0.3953803541837655
prediction train r2_score 0.4738247845946859
ETA test r2_score  -0.31549159768053325
prediction test r2_score 0.2616084612643903
ETA train MAPE 0.2563835743090772
prediction train MAPE 0.1843560061832306
ETA test MAPE  0.307574922241658
prediction test MAP